# 돌리고 나면 "전처리완료_나연.csv 생길 것임"

In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd
import json
from pandas import json_normalize
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geoband.API import *
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopandas import GeoDataFrame
from folium import Choropleth
import matplotlib
from datetime import datetime 
# GetCompasData로 주피터 노트북 경로에 파일 저장
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
matplotlib.rcParams['font.family']="Malgun Gothic"

In [2]:
# 서울
서울=["강남구","강동구","강북구","강서구","관악구","광진구","구로구","금천구","노원구","도봉구"
,"동대문구","동작구","마포구","서대문구","서초구","성동구","성북구","송파구","양천구","영등포구"
,"용산구","은평구","중구","중랑구", "종로구"]

# 수원
수원=["장안구","팔달구","권선구","영통구"]

# 창원
창원=["의창구", "성산구", "마산회원구", "마산합포구", "진해구"]

# 진주
진주=["진주시"]

# 전체
전체=["강남구","강동구","강북구","강서구","관악구","광진구","구로구","금천구","노원구","도봉구", 
    "동대문구","동작구","마포구","서대문구","서초구","성동구","성북구","송파구","양천구","영등포구",
    "용산구","은평구","중구","중랑구", "종로구", "장안구","팔달구","권선구","영통구", "의창구", "성산구", 
    "마산회원구", "마산합포구", "진해구", "진주시"]

In [3]:
# 행정구역 경계

SIG = gpd.read_file("전국시군구_경계구역.json")
HJD = gpd.read_file("전국행정동_경계구역.geojson")

SIG_4 = SIG[(SIG["SIG_CD"].str[:2] == "11") | 
            (SIG["SIG_KOR_NM"].str.split(" ").str[0] == "수원시") | 
            (SIG["SIG_KOR_NM"].str.split(" ").str[0] == "창원시") |
            (SIG["SIG_KOR_NM"].str.split(" ").str[0] == "진주시")]
# 서울 수원 창원 진주의 시군구 경계구역만 가져오기

SIG_4["sgg_nm"] =  SIG_4["SIG_KOR_NM"].apply(lambda x : x.split(" ")[-1])
# '창원시 의창구 -> 의창구'

HJD_4 = HJD[(HJD["sido"] == "11") |
        (HJD["sggnm"] == "수원시장안구") | (HJD["sggnm"] == "수원시권선구") | (HJD["sggnm"] == "수원시팔달구") | 
        (HJD["sggnm"] == "수원시영통구") | (HJD["sggnm"] == "진주시") | (HJD["sggnm"] == "창원시의창구") | (HJD["sggnm"] == "창원시성산구") |
        (HJD["sggnm"] == "창원시마산합포구") | (HJD["sggnm"] == "창원시마산회원구")| (HJD["sggnm"] == "창원시진해구")]

# 서울 수원 창원 진주의 행정동 경계구역만 가져오기

In [4]:
# 경찰서 경계
others_police_area = gpd.read_file('11.4개시도_경찰서_관할경계.geojson')
others_police_area.columns = ["관할경찰서","관할경찰청","geometry"]

경찰서=list(others_police_area['관할경찰서'].unique())

def point_police(data_df,others_police_area,col):
    df = data_df.copy()

    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    others_police_area.columns = ["관할경찰서","관할경찰청","geometry"]
    police_result = gpd.sjoin(others_police_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    
    police_result_groupby = police_result.groupby("관할경찰서").agg({col:"count"}).reset_index().rename(columns={col:"cnt"})
    
    return police_result,police_result_groupby

In [5]:
경찰서

['진주경찰서',
 '창원서부경찰서',
 '창원중부경찰서',
 '마산동부경찰서',
 '마산중부경찰서',
 '진해경찰서',
 '서울강동경찰서',
 '서울송파경찰서',
 '서울수서경찰서',
 '서울방배경찰서',
 '서울남대문경찰서',
 '서울서부경찰서',
 '서울양천경찰서',
 '서울강서경찰서',
 '서울성북경찰서',
 '서울용산경찰서',
 '서울광진경찰서',
 '서울종암경찰서',
 '수원남부경찰서',
 '서울동대문경찰서',
 '서울성동경찰서',
 '서울중부경찰서',
 '서울혜화경찰서',
 '서울중랑경찰서',
 '서울강남경찰서',
 '서울서초경찰서',
 '서울서대문경찰서',
 '서울종로경찰서',
 '서울마포경찰서',
 '수원서부경찰서',
 '수원중부경찰서',
 '서울관악경찰서',
 '서울동작경찰서',
 '서울금천경찰서',
 '서울구로경찰서',
 '서울영등포경찰서',
 '서울도봉경찰서',
 '서울은평경찰서',
 '서울강북경찰서',
 '서울노원경찰서']

In [6]:
# 원래 데이터
suwon_building = pd.read_csv('26.수원시_건축물대장(표제부).csv')
others_park = pd.read_csv('27.4개시도_공원현황.csv')
others_one_person = pd.read_csv('33.4개시도_1인가구수.csv')
others_basic_recipient = pd.read_csv('34.4개시도_기초수급자현황.csv')
others_sex_age_pop = pd.read_csv('35.4개시도_성연령별_총인구분포.csv')
others_foreigner_pop = pd.read_csv('36.4개시도_인구데이터(외국인).csv')
others_suicide = pd.read_csv('37.4개시도_자살율.csv')
others_fire = pd.read_csv('38.4개시도_화재발생통계.csv')
others_police = pd.read_csv('39.4개시도_경찰관서현황.csv')
others_playground = pd.read_csv('40.4개시도_어린이놀이시설현황.csv')
others_land_use_plan = gpd.read_file('41.4개시도_토지이용계획도.geojson')
others_convenience = gpd.read_file('42.4개시도_편의점현황.geojson')

# 전처리_나연.ipynb에서 추가한 데이터
others_park_sggnm=pd.read_csv('27.4개시도_공원현황_구별종류별.csv')
others_basic_recipient_age_range=pd.read_csv('34.4개시도_기초수급자현황_연령대.csv')
others_sex_age_pop_range=pd.read_csv('35.4개시도_성연령별_총인구분포_연령대.csv')
others_sex_age_pop_prod=pd.read_csv('35.4개시도_성연령별_총인구분포_생산연령.csv')
others_sex_age_pop_stat=pd.read_csv('35.4개시도_성연령별_총인구분포_통계값.csv')
others_police_sggnm=pd.read_csv('39.4개시도_경찰관서현황_구별.csv')
others_playground_sggnm=pd.read_csv('40.4개시도_어린이놀이시설현황_구별종류별.csv')
others_convenience_company=pd.read_csv("42.4개시도_편의점현황_업체별수.csv")

In [7]:
# 추가 데이터(이전 대회)
car_accident=pd.read_csv('11.5개시도_교통사고현황.csv')
police_num=pd.read_csv('12.경찰서별_지구대별_인원현황.csv')
safe_feeling=pd.read_csv('4.체감안전도_통계(2017~2019).csv')
crime_commit=pd.read_csv('6.범죄발생원표.csv')
crime_caught=pd.read_csv('7.범죄검거원표.csv')
crimial=pd.read_csv('8.범죄피의자원표.csv')
latest_school=pd.read_csv('19.5개시도_최종학력통계.csv')

## 데이터 프레임

In [8]:
police_4sido=pd.DataFrame([0]*len(경찰서), columns=["지역"], index=경찰서)
police_4sido.reset_index(inplace=True)
police_4sido.rename(columns={"index":"관할경찰서"}, inplace=True)
police_4sido.sort_values(by=["관할경찰서"], inplace=True)
police_4sido.reset_index(inplace=True)
police_4sido.drop(['index'], axis=1, inplace=True)
police_4sido['지역']=["창원시", "창원시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시",
                    "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", 
                    "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", "서울시", 
                   "서울시", "서울시", "서울시", "수원시", "수원시", "수원시", "진주시", "창원시", "창원시", "창원시"]
police_4sido

,관할경찰서,지역
0,마산동부경찰서,창원시
1,마산중부경찰서,창원시
2,서울강남경찰서,서울시
3,서울강동경찰서,서울시
4,서울강북경찰서,서울시
5,서울강서경찰서,서울시
6,서울관악경찰서,서울시
7,서울광진경찰서,서울시
8,서울구로경찰서,서울시
9,서울금천경찰서,서울시


# 제공 데이터
## 27.4개시도_공원현황/27.4개시도_공원현황_구별종류별



others_park = pd.read_csv('27.4개시도_공원현황.csv')

others_park_sggnm=pd.read_csv('27.4개시도_공원현황_구별종류별.csv')


In [9]:
others_park

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry,sgg_nm,park_nm,park_gbn,lon,lat
0,41111,"Jangan-gu, Suwon-si",수원시 장안구,"POLYGON ((127.0316846560563 37.33658806863813,...",장안구,9호소공원,소공원,126.966678,37.298179
1,41111,"Jangan-gu, Suwon-si",수원시 장안구,"POLYGON ((127.0316846560563 37.33658806863813,...",장안구,버들,어린이공원,126.982953,37.289519
2,41111,"Jangan-gu, Suwon-si",수원시 장안구,"POLYGON ((127.0316846560563 37.33658806863813,...",장안구,정천,어린이공원,126.980554,37.291157
3,41111,"Jangan-gu, Suwon-si",수원시 장안구,"POLYGON ((127.0316846560563 37.33658806863813,...",장안구,화산,어린이공원,126.980554,37.291157
4,41111,"Jangan-gu, Suwon-si",수원시 장안구,"POLYGON ((127.0316846560563 37.33658806863813,...",장안구,천천3,근린공원,126.986498,37.290738
...,...,...,...,...,...,...,...,...,...
2703,11740,Gangdong-gu,강동구,"POLYGON ((127.1783831276707 37.57041284347984,...",강동구,천일공원,어린이공원,127.136263,37.544989
2704,11740,Gangdong-gu,강동구,"POLYGON ((127.1783831276707 37.57041284347984,...",강동구,천호큰골목공원,어린이공원,127.138573,37.545101
2705,11740,Gangdong-gu,강동구,"POLYGON ((127.1783831276707 37.57041284347984,...",강동구,고분다리공원,어린이공원,127.131821,37.545405
2706,11740,Gangdong-gu,강동구,"POLYGON ((127.1783831276707 37.57041284347984,...",강동구,천호삼거리공원,어린이공원,127.138494,37.545489


In [10]:
others_park_result, others_park_groupby=point_police(others_park,others_police_area,'park_gbn')
others_park_result=others_park_result.groupby(['관할경찰서', 'park_gbn']).count()
others_park_result.reset_index(inplace=True)
others_park_result=others_park_result[['관할경찰서','park_gbn','park_nm']]
others_park_result

,관할경찰서,park_gbn,park_nm
0,마산동부경찰서,근린공원,11
1,마산동부경찰서,소공원,9
2,마산동부경찰서,어린이공원,44
3,마산중부경찰서,근린공원,16
4,마산중부경찰서,문화공원,1
...,...,...,...
158,창원중부경찰서,근린공원,28
159,창원중부경찰서,묘지공원,2
160,창원중부경찰서,소공원,12
161,창원중부경찰서,어린이공원,92


In [11]:
police_4sido['전체공원']=others_park_groupby['cnt']
police_4sido['어린이공원']=0
police_4sido['근린공원']=0
police_4sido['소공원']=0
police_4sido['문화공원']=0
police_4sido['역사공원']=0
police_4sido['체육공원']=0

In [12]:
for i in range(len(others_park_result)):
    for j in range(len(police_4sido)):
        if others_park_result.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_park_result.loc[i, 'park_gbn'] == '근린공원':
            police_4sido.loc[j, '근린공원']=others_park_result.loc[i, 'park_nm']
        elif others_park_result.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_park_result.loc[i, 'park_gbn'] == '소공원':
            police_4sido.loc[j, '소공원']=others_park_result.loc[i, 'park_nm']
        elif others_park_result.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_park_result.loc[i, 'park_gbn'] == '어린이공원':
            police_4sido.loc[j, '어린이공원']=others_park_result.loc[i, 'park_nm']
        elif others_park_result.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_park_result.loc[i, 'park_gbn'] == '문화공원':
            police_4sido.loc[j, '문화공원']=others_park_result.loc[i, 'park_nm']
        elif others_park_result.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_park_result.loc[i, 'park_gbn'] == '역사공원':
            police_4sido.loc[j, '역사공원']=others_park_result.loc[i, 'park_nm']
        elif others_park_result.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_park_result.loc[i, 'park_gbn'] == '체육공원':
            police_4sido.loc[j, '체육공원']=others_park_result.loc[i, 'park_nm']


In [13]:
police_4sido

,관할경찰서,지역,전체공원,어린이공원,근린공원,소공원,문화공원,역사공원,체육공원
0,마산동부경찰서,창원시,64,44,11,9,0,0,0
1,마산중부경찰서,창원시,55,36,16,2,1,0,0
2,서울강남경찰서,서울시,24,12,10,0,1,1,0
3,서울강동경찰서,서울시,72,53,18,1,0,0,0
4,서울강북경찰서,서울시,41,31,2,5,1,0,0
5,서울강서경찰서,서울시,139,117,22,0,0,0,0
6,서울관악경찰서,서울시,78,65,5,8,0,0,0
7,서울광진경찰서,서울시,34,29,0,5,0,0,0
8,서울구로경찰서,서울시,41,22,8,9,2,0,0
9,서울금천경찰서,서울시,50,40,8,2,0,0,0


## 33.4개시도_1인가구수


others_one_person = pd.read_csv('33.4개시도_1인가구수.csv')


In [14]:
others_one_person

,sido,sgg_nm,sx,age,date,single_person_hh,house_total_num,detached_house,apartment,townhouse,multiplex_house,house_in_commercial,not_in_house
0,서울특별시,종로구,계,합계,2017,23638,19260,11234,2992,887,3279,868,4378
1,서울특별시,종로구,계,20세 미만,2017,507,459,379,11,11,38,20,48
2,서울특별시,종로구,계,20~24,2017,2992,2519,1773,158,44,340,204,473
3,서울특별시,종로구,계,25~29,2017,3627,2845,1677,398,77,552,141,782
4,서울특별시,종로구,계,30~34,2017,2552,1970,1024,357,64,449,76,582
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,서울특별시,광진구,여자,65~69,2019,1401,1371,837,202,39,257,36,30
5036,서울특별시,광진구,여자,70~74,2019,1300,1268,783,164,35,254,32,32
5037,서울특별시,광진구,여자,75~79,2019,1221,1182,740,175,44,197,26,39
5038,서울특별시,광진구,여자,80~84,2019,849,802,471,170,20,122,19,47


## 34.4개시도_기초수급자현황/34.4개시도_기초수급자현황_연령대


others_basic_recipient = pd.read_csv('34.4개시도_기초수급자현황.csv')

others_basic_recipient_age_range=pd.read_csv('34.4개시도_기초수급자현황_연령대.csv')


In [15]:
others_basic_recipient

,date,sido,sgg_nm,age,num,age_range
0,201706,서울특별시,종로구,4,4,0대
1,201706,서울특별시,종로구,45,26,40대
2,201706,서울특별시,종로구,46,55,40대
3,201706,서울특별시,종로구,47,37,40대
4,201706,서울특별시,종로구,48,51,40대
...,...,...,...,...,...,...
140599,202105,경상남도,진주시,39,94,30대
140600,202105,경상남도,진주시,40,122,40대
140601,202105,경상남도,진주시,41,135,40대
140602,202105,경상남도,진주시,42,114,40대


## 35.4개시도_성연령별_총인구분포
- 35.4개시도_성연령별_총인구분포_연령대
- 35.4개시도_성연령별_총인구분포_생산연령
- 35.4개시도_성연령별_총인구분포_통계값



others_sex_age_pop = pd.read_csv('35.4개시도_성연령별_총인구분포.csv')

others_sex_age_pop_range=pd.read_csv('35.4개시도_성연령별_총인구분포_연령대.csv')

others_sex_age_pop_prod=pd.read_csv('35.4개시도_성연령별_총인구분포_생산연령.csv')

others_sex_age_pop_stat=pd.read_csv('35.4개시도_성연령별_총인구분포_통계값.csv')


In [16]:
others_sex_age_pop

,sido,sgg_nm,age,date,popu_num,popu_male_num,popu_female_num,popu_sx_rate,local_num,local_male_num,local_female_num,local_sx_rate
0,서울특별시,종로구,합계,2017,157277.0,76670.0,80607.0,95.1,146298.0,71757.0,74541.0,96.3
1,서울특별시,종로구,0~4세,2017,4137.0,2099.0,2038.0,103.0,4011.0,2039.0,1972.0,103.4
2,서울특별시,종로구,0세,2017,694.0,336.0,358.0,93.9,675.0,327.0,348.0,94.0
3,서울특별시,종로구,1세,2017,836.0,429.0,407.0,105.4,805.0,412.0,393.0,104.8
4,서울특별시,종로구,2세,2017,878.0,459.0,419.0,109.5,850.0,447.0,403.0,110.9
...,...,...,...,...,...,...,...,...,...,...,...,...
13435,서울특별시,중구,15세미만,2017,11035.0,5604.0,5431.0,103.2,10565.0,5334.0,5231.0,102.0
13436,서울특별시,중구,15~64세,2017,97649.0,48332.0,49317.0,98.0,88374.0,44025.0,44349.0,99.3
13437,서울특별시,중구,65세이상,2017,19212.0,8259.0,10953.0,75.4,18692.0,8032.0,10660.0,75.3
13438,서울특별시,중구,평균연령,2017,42.8,42.1,43.5,NaN,43.2,42.4,43.9,NaN


## 36.4개시도_인구데이터(외국인)


others_foreigner_pop = pd.read_csv('36.4개시도_인구데이터(외국인).csv')


In [17]:
others_foreigner_pop

,sido,sgg_nm,foreigner_all(2017),foreigner_M(2017),foreigner_F(2017),foreigner_all(2018),foreigner_M(2018),foreigner_F(2018),foreigner_all(2019),foreigner_M(2019),foreigner_F(2019)
0,서울특별시,종로구,10979,4913,6066,12056,5536,6520,13159,5818,7341
1,서울특별시,중구,10265,4804,5461,11988,5735,6253,12559,6025,6534
2,서울특별시,용산구,16005,8712,7293,19126,10879,8247,20503,11621,8882
3,서울특별시,성동구,10436,5042,5394,11307,5403,5904,11774,5610,6164
4,서울특별시,광진구,18399,8323,10076,19729,9032,10697,20925,9474,11451
5,서울특별시,동대문구,17942,7483,10459,19848,8324,11524,21049,8852,12197
6,서울특별시,중랑구,6013,2475,3538,6586,2728,3858,7091,2962,4129
7,서울특별시,성북구,13191,5694,7497,14523,6173,8350,15552,6621,8931
8,서울특별시,강북구,4443,1719,2724,4947,1971,2976,5346,2088,3258
9,서울특별시,도봉구,2629,1059,1570,2951,1199,1752,3320,1380,1940


## 37.4개시도_자살율


others_suicide = pd.read_csv('37.4개시도_자살율.csv')


In [18]:
others_suicide

,sido,sgg_nm,sx,dead_num_2017,dptt_2017,age_stndrd_dptt_2017,dead_num_2018,dptt_2018,age_stndrd_dptt_2018,dead_num_2019,dptt_2019,age_stndrd_dptt_2019
0,서울특별시,종로구,남자,33,45.3,37.2,22,30.3,28.7,30,41.9,32.0
1,서울특별시,종로구,여자,12,15.9,12.4,8,10.6,9.6,9,12.0,11.4
2,서울특별시,중구,남자,20,33.4,26.0,17,28.4,20.0,20,33.4,28.5
3,서울특별시,중구,여자,7,11.5,6.4,7,11.4,9.7,5,8.1,8.8
4,서울특별시,용산구,남자,36,33.7,29.8,33,31.0,26.8,27,25.4,19.6
...,...,...,...,...,...,...,...,...,...,...,...,...
65,창원시,마산합포구,여자,13,14.4,11.6,14,15.6,16.5,10,11.2,11.7
66,창원시,마산회원구,남자,30,28.9,24.7,33,32.7,27.9,40,40.7,35.0
67,창원시,마산회원구,여자,7,6.9,7.1,15,15.1,13.4,15,15.5,18.8
68,창원시,진해구,남자,24,25.0,22.5,39,40.1,36.1,37,37.6,32.3


## 38.4개시도_화재발생통계



others_fire = pd.read_csv('38.4개시도_화재발생통계.csv')


In [19]:
others_fire

,yr,sido,sgg_nm,acc_num,dead_num,injr_num,tot_vic_num,prop_dmg,chattel_dmg,assts_dmg,dmg_per_acc
0,2018,경기도,권선구,108,0,18,18,1034047,756064,1790111,16575
1,2018,경기도,영통구,89,1,2,3,302241,466660,768901,8639
2,2018,경기도,장안구,92,0,4,4,203169,200131,403300,4384
3,2018,경기도,팔달구,81,0,17,17,941048,1031151,1972199,24348
4,2019,경기도,권선구,124,0,11,11,331803,472602,804405,6487
...,...,...,...,...,...,...,...,...,...,...,...
100,2020,서울특별시,용산구,192,0,5,5,64235,205600,269835,1405
101,2020,서울특별시,은평구,192,2,6,8,270762,234026,504788,2629
102,2020,서울특별시,종로구,217,2,5,7,192633,447118,639751,2948
103,2020,서울특별시,중구,185,0,10,10,573942,710480,1284422,6943


## 39.4개시도_경찰관서현황/39.4개시도_경찰관서현황_구별


others_police = pd.read_csv('39.4개시도_경찰관서현황.csv')

others_police_sggnm=pd.read_csv('39.4개시도_경찰관서현황_구별.csv')


In [20]:
others_police_result, others_police_groupby=point_police(others_police,others_police_area,'name')
police_4sido['경찰관서']=others_police_groupby['cnt']

## 40.4개시도_어린이놀이시설현황/40.4개시도_어린이놀이시설현황_구별종류별


others_playground = pd.read_csv('40.4개시도_어린이놀이시설현황.csv')

others_playground_sggnm=pd.read_csv('40.4개시도_어린이놀이시설현황_구별종류별.csv')



In [21]:
others_playground

,fac_no,fac_nm,fac_ins_date,fac_loc,addr,lat,lon,건설이후(일),sgg_nm
0,500536,진주초등학교 어린이놀이시설,1895-09-24,학교,경상남도 진주시 진주성로24번길 7 (인사동),35.193069,128.078342,46029,진주시
1,539182,봉래초등학교 어린이놀이시설,1910-07-01,학교,경상남도 진주시 의병로 265 (봉래동),35.199850,128.087968,40636,진주시
2,37623,금곡초등학교 어린이놀이시설,1923-09-10,학교,경상남도 진주시 금곡면 월아산로89번길 11,35.089813,128.184974,35817,진주시
3,500509,봉원초등학교 어린이놀이시설,1967-05-31,학교,경상남도 진주시 의병로 95-20 (상봉동),35.201066,128.069919,19848,진주시
4,500126,관봉초등학교 어린이놀이시설,1970-07-01,학교,경상남도 진주시 정촌면 정촌로 629,35.137041,128.143664,18721,진주시
...,...,...,...,...,...,...,...,...,...
13088,560787,중계주공9단지 복지동 옆 어린이놀이터,2017-07-18,주택단지,서울특별시 노원구 노원로16길 15 (하계동 중계주공9단지아파트),37.642858,127.073979,1537,노원구
13089,561658,불암산(삼육대) 유아숲체험원,2017-10-19,도시공원,서울 노원구 공릉동산 223-1번지 삼육대 제명호 주변,37.642555,127.099103,1444,노원구
13090,562586,중계주공9단지 노인복지관 옆 어린이놀이터,2017-12-29,주택단지,서울특별시 노원구 노원로16길 15 (하계동 중계주공9단지아파트),37.642858,127.073979,1373,노원구
13091,564803,당고개지구공원 놀이터(물놀이장),2018-08-01,도시공원,서울특별시 노원구 상계동113-2번지 수락산 당고개지구공원,37.669064,127.077097,1158,노원구


In [22]:
others_playground_result, others_playground_groupby=point_police(others_playground,others_police_area,'fac_loc')
others_playground_result_=others_playground_result.groupby(['관할경찰서', 'fac_loc']).count()
others_playground_result_.reset_index(inplace=True)
others_playground_result_=others_playground_result_[['관할경찰서','fac_loc', 'fac_nm']]
others_playground_result_.rename(columns={'fac_nm':'count'}, inplace=True)
others_playground_result_

,관할경찰서,fac_loc,count
0,마산동부경찰서,놀이제공영업소,3
1,마산동부경찰서,대규모점포,1
2,마산동부경찰서,도시공원,44
3,마산동부경찰서,식품접객업소,8
4,마산동부경찰서,아동복지시설,1
...,...,...,...
396,창원중부경찰서,유치원,21
397,창원중부경찰서,의료기관,1
398,창원중부경찰서,종교시설,1
399,창원중부경찰서,주택단지,171


In [23]:
police_4sido['어린이놀이시설']=others_playground_groupby['cnt']
police_4sido['어린이집']=0
police_4sido['주택단지']=0
police_4sido['유치원']=0
police_4sido['학교']=0
police_4sido['도시공원']=0
police_4sido['식품접객업소']=0
police_4sido['놀이제공영업소']=0
police_4sido['종교시설']=0
police_4sido['아동복지시설']=0
police_4sido['대규모점포']=0

In [24]:
for i in range(len(others_playground_result_)):
    for j in range(len(police_4sido)):
        if others_playground_result_.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_playground_result_.loc[i, 'fac_loc'] == '어린이집':
            police_4sido.loc[j, '어린이집']=others_playground_result_.loc[i, 'count']
        elif others_playground_result_.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_playground_result_.loc[i, 'fac_loc'] == '주택단지':
            police_4sido.loc[j, '주택단지']=others_playground_result_.loc[i, 'count']
        elif others_playground_result_.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_playground_result_.loc[i, 'fac_loc'] == '유치원':
            police_4sido.loc[j, '유치원']=others_playground_result_.loc[i, 'count']
        elif others_playground_result_.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_playground_result_.loc[i, 'fac_loc'] == '학교':
            police_4sido.loc[j, '학교']=others_playground_result_.loc[i, 'count']
        elif others_playground_result_.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_playground_result_.loc[i, 'fac_loc'] == '도시공원':
            police_4sido.loc[j, '도시공원']=others_playground_result_.loc[i, 'count']
        elif others_playground_result_.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_playground_result_.loc[i, 'fac_loc'] == '식품접객업소':
            police_4sido.loc[j, '식품접객업소']=others_playground_result_.loc[i, 'count']
        elif others_playground_result_.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_playground_result_.loc[i, 'fac_loc'] == '놀이제공영업소':
            police_4sido.loc[j, '놀이제공영업소']=others_playground_result_.loc[i, 'count']
        elif others_playground_result_.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_playground_result_.loc[i, 'fac_loc'] == '종교시설':
            police_4sido.loc[j, '종교시설']=others_playground_result_.loc[i, 'count']
        elif others_playground_result_.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_playground_result_.loc[i, 'fac_loc'] == '아동복지시설':
            police_4sido.loc[j, '아동복지시설']=others_playground_result_.loc[i, 'count']
        elif others_playground_result_.loc[i, '관할경찰서'] == police_4sido.loc[j, '관할경찰서'] and others_playground_result_.loc[i, 'fac_loc'] == '대규모점포':
            police_4sido.loc[j, '대규모점포']=others_playground_result_.loc[i, 'count']

In [25]:
others_playground_result_days=others_playground_result.groupby(['관할경찰서', 'fac_loc']).sum()
others_playground_result_days.reset_index(inplace=True)
others_playground_result_days=others_playground_result_days[['관할경찰서','건설이후(일)']]
others_playground_result_days_=others_playground_result_days.groupby(['관할경찰서']).sum()
others_playground_result_days_.reset_index(inplace=True)
police_4sido['건설이후(일)']=others_playground_result_days_['건설이후(일)']
others_playground_result_days_

,관할경찰서,건설이후(일)
0,마산동부경찰서,1507340
1,마산중부경찰서,1126729
2,서울강남경찰서,1182124
3,서울강동경찰서,1788883
4,서울강북경찰서,1049841
5,서울강서경찰서,2598175
6,서울관악경찰서,1623401
7,서울광진경찰서,1348329
8,서울구로경찰서,2151029
9,서울금천경찰서,879015


In [26]:
police_4sido

,관할경찰서,지역,전체공원,어린이공원,근린공원,소공원,문화공원,역사공원,체육공원,경찰관서,...,주택단지,유치원,학교,도시공원,식품접객업소,놀이제공영업소,종교시설,아동복지시설,대규모점포,건설이후(일)
0,마산동부경찰서,창원시,64,44,11,9,0,0,0,8,...,137,30,21,44,8,3,1,1,1,1507340
1,마산중부경찰서,창원시,55,36,16,2,1,0,0,9,...,131,14,19,11,5,0,0,0,2,1126729
2,서울강남경찰서,서울시,24,12,10,0,1,1,0,9,...,138,8,9,24,2,3,1,1,0,1182124
3,서울강동경찰서,서울시,72,53,18,1,0,0,0,10,...,210,21,22,59,5,3,0,8,1,1788883
4,서울강북경찰서,서울시,41,31,2,5,1,0,0,9,...,109,11,18,52,4,7,2,1,0,1049841
5,서울강서경찰서,서울시,139,117,22,0,0,0,0,11,...,335,40,35,123,8,11,1,1,8,2598175
6,서울관악경찰서,서울시,78,65,5,8,0,0,0,10,...,180,27,19,80,3,4,1,0,1,1623401
7,서울광진경찰서,서울시,34,29,0,5,0,0,0,12,...,100,13,25,43,3,5,0,0,0,1348329
8,서울구로경찰서,서울시,41,22,8,9,2,0,0,9,...,287,21,29,37,5,4,3,3,1,2151029
9,서울금천경찰서,서울시,50,40,8,2,0,0,0,6,...,91,12,16,48,1,3,0,2,3,879015


## 42.4개시도_편의점현황/42.4개시도_편의점현황_업체별수


others_convenience = gpd.read_file('42.4개시도_편의점현황.geojson')

others_convenience_company=pd.read_csv("42.4개시도_편의점현황_업체별수.csv")

In [27]:
others_convenience

,STORE_ID,BUSINESS_NM,KSIC_NM,SIDO_NM,sgg_nm,ADB_EMD_NM,EMD_NM,LAND_GBN_NM,JIBUN_ADDR,ROAD_NM,BLDG_ID,ROAD_ADDR,lon,lat,COMPANY,geometry
0,12033797,GS25인계타운,체인화 편의점,경기도,팔달구,인계동,인계동,대지,경기도 수원시 팔달구 인계동 1031-6,경기도 수원시 팔달구 효원로249번길,4111514100110310006011650,경기도 수원시 팔달구 효원로249번길 47,127.029764,37.264680,GS25,POINT (127.02976 37.26468)
1,12039403,CU 장안연무점,체인화 편의점,경기도,장안구,연무동,연무동,대지,경기도 수원시 장안구 연무동 260-33,경기도 수원시 장안구 창룡대로151번길,4111113700102600033000024,경기도 수원시 장안구 창룡대로151번길 2,127.026622,37.291552,CU,POINT (127.02662 37.29155)
2,11812683,세븐일레븐오목천대로점 오목천대로점,체인화 편의점,경기도,권선구,평동,오목천동,대지,경기도 수원시 권선구 오목천동 558-7,경기도 수원시 권선구 오목천로,4111312900105580007034976,경기도 수원시 권선구 오목천로 54,126.968346,37.239543,세븐일레븐,POINT (126.96835 37.23954)
3,12033749,GS25탑동중앙탑동중앙점 탑동중앙점,체인화 편의점,경기도,권선구,서둔동,탑동,대지,경기도 수원시 권선구 탑동 783-15,경기도 수원시 권선구 탑동로39번길,4111313300107830015025390,경기도 수원시 권선구 탑동로39번길 1,126.974169,37.272285,GS25,POINT (126.97417 37.27229)
4,20320119,미니스톱 영화북문점,체인화 편의점,경기도,장안구,영화동,영화동,대지,경기도 수원시 장안구 영화동 326-6,경기도 수원시 장안구 정조로921번길,4111113400103260006036857,경기도 수원시 장안구 정조로921번길 5,127.012999,37.289745,미니스톱,POINT (127.01300 37.28974)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10997,25238608,세븐일레븐 롯데몰수원역점,체인화 편의점,경기도,권선구,서둔동,서둔동,대지,경기도 수원시 권선구 서둔동 296-77,경기도 수원시 권선구 세화로,4111313100102960077000001,경기도 수원시 권선구 세화로 134,126.997278,37.264166,세븐일레븐,POINT (126.99728 37.26417)
10998,16025943,이마트 24수원행궁점,체인화 편의점,경기도,팔달구,행궁동,남창동,대지,경기도 수원시 팔달구 남창동 110,경기도 수원시 팔달구 행궁로,4111512300101100000011185,경기도 수원시 팔달구 행궁로 41,127.015374,37.278825,이마트,POINT (127.01537 37.27883)
10999,11809397,세븐일레븐 팔달중동점,체인화 편의점,경기도,팔달구,행궁동,중동,대지,경기도 수원시 팔달구 중동 33,경기도 수원시 팔달구 중부대로,4111512500100330000005176,경기도 수원시 팔달구 중부대로 6,127.017465,37.274995,세븐일레븐,POINT (127.01746 37.27500)
11000,24441852,GS25 광교센타점,체인화 편의점,경기도,영통구,광교2동,하동,대지,경기도 수원시 영통구 하동 972-1,경기도 수원시 영통구 법조로149번길,4111710400109720001000001,경기도 수원시 영통구 법조로149번길 171-3,127.069666,37.292844,GS25,POINT (127.06967 37.29284)


In [28]:
others_convenience_result, others_convenience_groupby=point_police(others_convenience,others_police_area,'KSIC_NM')
others_convenience_result=others_convenience_result.groupby(['관할경찰서', 'KSIC_NM']).count()
others_convenience_result.reset_index(inplace=True)
others_convenience_result=others_convenience_result[['관할경찰서','KSIC_NM','BUSINESS_NM']]
others_convenience_result

,관할경찰서,KSIC_NM,BUSINESS_NM
0,마산동부경찰서,체인화 편의점,198
1,마산중부경찰서,그외 기타 음식점업,1
2,마산중부경찰서,체인화 편의점,147
3,서울강남경찰서,체인화 편의점,396
4,서울강동경찰서,기타 대형 종합 소매업,1
5,서울강동경찰서,체인화 편의점,348
6,서울강북경찰서,체인화 편의점,216
7,서울강서경찰서,기타 대형 종합 소매업,1
8,서울강서경찰서,체인화 편의점,500
9,서울관악경찰서,체인화 편의점,418


In [29]:
## 그외 기타 종합 소매업, 대형 종합 소매업 등 무시
police_4sido['편의점']=others_convenience_groupby['cnt']

In [30]:
police_4sido

,관할경찰서,지역,전체공원,어린이공원,근린공원,소공원,문화공원,역사공원,체육공원,경찰관서,...,유치원,학교,도시공원,식품접객업소,놀이제공영업소,종교시설,아동복지시설,대규모점포,건설이후(일),편의점
0,마산동부경찰서,창원시,64,44,11,9,0,0,0,8,...,30,21,44,8,3,1,1,1,1507340,198
1,마산중부경찰서,창원시,55,36,16,2,1,0,0,9,...,14,19,11,5,0,0,0,2,1126729,148
2,서울강남경찰서,서울시,24,12,10,0,1,1,0,9,...,8,9,24,2,3,1,1,0,1182124,396
3,서울강동경찰서,서울시,72,53,18,1,0,0,0,10,...,21,22,59,5,3,0,8,1,1788883,349
4,서울강북경찰서,서울시,41,31,2,5,1,0,0,9,...,11,18,52,4,7,2,1,0,1049841,216
5,서울강서경찰서,서울시,139,117,22,0,0,0,0,11,...,40,35,123,8,11,1,1,8,2598175,501
6,서울관악경찰서,서울시,78,65,5,8,0,0,0,10,...,27,19,80,3,4,1,0,1,1623401,418
7,서울광진경찰서,서울시,34,29,0,5,0,0,0,12,...,13,25,43,3,5,0,0,0,1348329,371
8,서울구로경찰서,서울시,41,22,8,9,2,0,0,9,...,21,29,37,5,4,3,3,1,2151029,317
9,서울금천경찰서,서울시,50,40,8,2,0,0,0,6,...,12,16,48,1,3,0,2,3,879015,267


# 추가 데이터
## 4.체감안전도_통계(2017~2019)

safe_feeling=pd.read_csv('4.체감안전도_통계(2017~2019).csv')


In [31]:
safe_feeling

,large_category,medium_category,tot_f_safety,dept_safety,crime_safety,rob_satety_q1,mur_safety_q2,ta_safety_q3,raw_odder_q4,overall_q5
0,2017 상반기(연령대),10대(만14세 이상),74.3,73.3,81.5,80.1,82.8,70.9,66.9,76.5
1,2017 상반기(연령대),20대,70.1,69.3,76.9,75.7,78.2,66.1,64.3,71.7
2,2017 상반기(연령대),30대,69.6,68.9,76.8,75.7,77.9,63.9,65.2,71.1
3,2017 상반기(연령대),40대,70.7,70.2,77.2,76.1,78.3,65.5,67.0,71.9
4,2017 상반기(연령대),50대,72.1,71.5,78.3,77.7,79.0,68.0,67.6,73.3
...,...,...,...,...,...,...,...,...,...,...
285,2019 하반기(경찰서별),경남창원서부서,74.0,73.2,79.3,79.1,79.5,68.5,71.2,76.0
286,2019 하반기(경찰서별),경남마산중부서,75.4,74.9,80.4,80.8,80.0,70.8,73.2,76.5
287,2019 하반기(경찰서별),경남마산동부서,71.9,71.4,75.8,75.0,76.5,69.4,68.9,73.1
288,2019 하반기(경찰서별),경남진주서,75.1,74.9,79.2,78.7,79.6,70.3,74.6,75.6


In [32]:
safe_feeling["cat"]=safe_feeling["large_category"].apply(lambda x : x.split("(")[-1][:-1])
safe_feeling

,large_category,medium_category,tot_f_safety,dept_safety,crime_safety,rob_satety_q1,mur_safety_q2,ta_safety_q3,raw_odder_q4,overall_q5,cat
0,2017 상반기(연령대),10대(만14세 이상),74.3,73.3,81.5,80.1,82.8,70.9,66.9,76.5,연령대
1,2017 상반기(연령대),20대,70.1,69.3,76.9,75.7,78.2,66.1,64.3,71.7,연령대
2,2017 상반기(연령대),30대,69.6,68.9,76.8,75.7,77.9,63.9,65.2,71.1,연령대
3,2017 상반기(연령대),40대,70.7,70.2,77.2,76.1,78.3,65.5,67.0,71.9,연령대
4,2017 상반기(연령대),50대,72.1,71.5,78.3,77.7,79.0,68.0,67.6,73.3,연령대
...,...,...,...,...,...,...,...,...,...,...,...
285,2019 하반기(경찰서별),경남창원서부서,74.0,73.2,79.3,79.1,79.5,68.5,71.2,76.0,경찰서별
286,2019 하반기(경찰서별),경남마산중부서,75.4,74.9,80.4,80.8,80.0,70.8,73.2,76.5,경찰서별
287,2019 하반기(경찰서별),경남마산동부서,71.9,71.4,75.8,75.0,76.5,69.4,68.9,73.1,경찰서별
288,2019 하반기(경찰서별),경남진주서,75.1,74.9,79.2,78.7,79.6,70.3,74.6,75.6,경찰서별


## 6.범죄발생원표(2017-2019)


crime_commit=pd.read_csv('6.범죄발생원표.csv')


In [33]:
crime_commit

,jur_stn,crm,crm_wthr,crm_clue,vic_sx,vic_age,crm_loc,crm_tm,crm_date,5m_crm_yn
0,서울수서경찰서,위조외국통화행사,미상,진정,불상,,은행,09:00-11:59,20170101,
1,서울영등포경찰서,도로교통법위반,맑음,타인신고,불상,,노상,21:00-23:59,20170101,
2,서울양천경찰서,209015100,미상,피해자신고,남자,60세초과,노상,미상,20170101,
3,서울서초경찰서,폭행,미상,피해자신고,여자,40세이하,기타,21:00-23:59,20170101,폭력
4,서울동대문경찰서,사기,미상,진정,여자,30세이하,기타,미상,20170101,
...,...,...,...,...,...,...,...,...,...,...
1073217,경남진해경찰서,사기,미상,고소,2,60세이하,기타,미상,9/27/2019,
1073218,경남진해경찰서,폭행,맑음,피해자신고,3,,차안,00:00∼02:59,10/4/2019,폭력
1073219,경남마산동부경찰서,재물손괴,미상,피해자신고,2,60세이하,주택,12:00∼14:59,9/17/2019,폭력
1073220,경남마산중부경찰서,사기,미상,진정,1,50세이하,기타,12:00∼14:59,9/14/2019,


## 7.범죄검거원표(2017-2019)


crime_caught=pd.read_csv('7.범죄검거원표.csv')


In [34]:
crime_caught

,date,jur_stn,crm
0,201701,경남마산중부경찰서,사기
1,201701,경남마산중부경찰서,경범죄처벌법위반
2,201612,경남마산중부경찰서,도로교통법위반(음주운전)
3,201702,경남마산중부경찰서,위계공무집행방해
4,201702,경남마산중부경찰서,위계공무집행방해
...,...,...,...
928560,201912,경남창원중부경찰서,교통사고처리특례법위반(치상)
928561,201912,경남창원중부경찰서,사기
928562,201912,경남창원중부경찰서,사기
928563,201912,경남창원중부경찰서,사기


## 8.범죄피의자원표(2017-2019)


crimial=pd.read_csv('8.범죄피의자원표.csv')


In [35]:
crimial

,jur_stn,date,crm,ofn_age,ofn_sx,ofn_edu,ofn_sol
0,서울서초경찰서,201709,의료법위반,58,여,NaN,NaN
1,서울은평경찰서,201710,대외무역법위반,0,불상,NaN,NaN
2,서울은평경찰서,201710,대외무역법위반,52,여,대학졸업,상류
3,서울은평경찰서,201710,대외무역법위반,56,남,대학졸업,상류
4,서울남대문경찰서,201701,사기,59,여,미상,하류
...,...,...,...,...,...,...,...
1290331,경남마산동부경찰서,201908,사기,21,남,고등학교졸업,하류
1290332,경남마산동부경찰서,201906,사기,0,불상,NaN,NaN
1290333,경남마산중부경찰서,201906,사기,26,남,고등학교졸업,하류
1290334,경남마산중부경찰서,201910,저작권법위반,48,남,NaN,NaN


## 11.5개시도_교통사고현황(2018-2020)


car_accident=pd.read_csv('11.5개시도_교통사고현황.csv')


In [36]:
car_accident

,date,acc_case,ofn_sx,ofn_age,vic_sx,vic_age,jur_stn
0,2018년 1월,차대차 - 측면충돌,남,20대,남,50대,서울중부
1,2018년 1월,차대차 - 측면충돌,여,50대,남,70대,서울중부
2,2018년 1월,차대차 - 측면충돌,남,40대,남,50대,서울중부
3,2018년 1월,차대차 - 기타,기타불명,미분류,여,40대,서울중부
4,2018년 1월,차대차 - 기타,남,60대,남,50대,서울중부
...,...,...,...,...,...,...,...
140949,2019년 8월,차대사람 - 횡단중,남,40대,여,90대 이상,마산중부
140950,2020년 12월,차대사람 - 차도통행중,여,40대,여,90대 이상,창원중부
140951,2019년 4월,차대사람 - 횡단중,남,60대,남,90대 이상,진해경찰서
140952,2019년 5월,차대사람 - 차도통행중,남,20대,남,90대 이상,진해경찰서


## 12.경찰서별_지구대별_인원현황


police_num=pd.read_csv('12.경찰서별_지구대별_인원현황.csv')


In [37]:
police_num_groupby=police_num.groupby(['pol_office']).sum()
police_num_groupby.reset_index(inplace=True)
police_4sido['지구대인원']=police_num_groupby['num']
police_4sido

,관할경찰서,지역,전체공원,어린이공원,근린공원,소공원,문화공원,역사공원,체육공원,경찰관서,...,학교,도시공원,식품접객업소,놀이제공영업소,종교시설,아동복지시설,대규모점포,건설이후(일),편의점,지구대인원
0,마산동부경찰서,창원시,64,44,11,9,0,0,0,8,...,21,44,8,3,1,1,1,1507340,198,185
1,마산중부경찰서,창원시,55,36,16,2,1,0,0,9,...,19,11,5,0,0,0,2,1126729,148,142
2,서울강남경찰서,서울시,24,12,10,0,1,1,0,9,...,9,24,2,3,1,1,0,1182124,396,421
3,서울강동경찰서,서울시,72,53,18,1,0,0,0,10,...,22,59,5,3,0,8,1,1788883,349,393
4,서울강북경찰서,서울시,41,31,2,5,1,0,0,9,...,18,52,4,7,2,1,0,1049841,216,347
5,서울강서경찰서,서울시,139,117,22,0,0,0,0,11,...,35,123,8,11,1,1,8,2598175,501,500
6,서울관악경찰서,서울시,78,65,5,8,0,0,0,10,...,19,80,3,4,1,0,1,1623401,418,493
7,서울광진경찰서,서울시,34,29,0,5,0,0,0,12,...,25,43,3,5,0,0,0,1348329,371,380
8,서울구로경찰서,서울시,41,22,8,9,2,0,0,9,...,29,37,5,4,3,3,1,2151029,317,69
9,서울금천경찰서,서울시,50,40,8,2,0,0,0,6,...,16,48,1,3,0,2,3,879015,267,274


## 19.5개시도_최종학력통계

latest_school=pd.read_csv('19.5개시도_최종학력통계.csv')

In [38]:
latest_school

,sido,sgg_nm,emd_nm,sx,age,inv_edu,inv_detail,num_in_2015
0,서울특별시,종로구,사직동,남자,합계,계,6세 이상 내국인,3669.0
1,서울특별시,종로구,사직동,남자,합계,초등학교,6세 이상 내국인,339.0
2,서울특별시,종로구,사직동,남자,합계,중학교,6세 이상 내국인,237.0
3,서울특별시,종로구,사직동,남자,합계,고등학교,6세 이상 내국인,629.0
4,서울특별시,종로구,사직동,남자,합계,"대학교(2,3년제)",6세 이상 내국인,196.0
...,...,...,...,...,...,...,...,...
80339,경상남도,창원시진해구,웅동2동,여자,70세 이상,고등학교,6세 이상 내국인,57.0
80340,경상남도,창원시진해구,웅동2동,여자,70세 이상,"대학교(2,3년제)",6세 이상 내국인,8.0
80341,경상남도,창원시진해구,웅동2동,여자,70세 이상,대학교(4년제 이상),6세 이상 내국인,NaN
80342,경상남도,창원시진해구,웅동2동,여자,70세 이상,대학원(석박사 과정),6세 이상 내국인,NaN


In [39]:
police_4sido.to_csv("전처리완료_나연.csv", index=False)

In [40]:
police_4sido

,관할경찰서,지역,전체공원,어린이공원,근린공원,소공원,문화공원,역사공원,체육공원,경찰관서,...,학교,도시공원,식품접객업소,놀이제공영업소,종교시설,아동복지시설,대규모점포,건설이후(일),편의점,지구대인원
0,마산동부경찰서,창원시,64,44,11,9,0,0,0,8,...,21,44,8,3,1,1,1,1507340,198,185
1,마산중부경찰서,창원시,55,36,16,2,1,0,0,9,...,19,11,5,0,0,0,2,1126729,148,142
2,서울강남경찰서,서울시,24,12,10,0,1,1,0,9,...,9,24,2,3,1,1,0,1182124,396,421
3,서울강동경찰서,서울시,72,53,18,1,0,0,0,10,...,22,59,5,3,0,8,1,1788883,349,393
4,서울강북경찰서,서울시,41,31,2,5,1,0,0,9,...,18,52,4,7,2,1,0,1049841,216,347
5,서울강서경찰서,서울시,139,117,22,0,0,0,0,11,...,35,123,8,11,1,1,8,2598175,501,500
6,서울관악경찰서,서울시,78,65,5,8,0,0,0,10,...,19,80,3,4,1,0,1,1623401,418,493
7,서울광진경찰서,서울시,34,29,0,5,0,0,0,12,...,25,43,3,5,0,0,0,1348329,371,380
8,서울구로경찰서,서울시,41,22,8,9,2,0,0,9,...,29,37,5,4,3,3,1,2151029,317,69
9,서울금천경찰서,서울시,50,40,8,2,0,0,0,6,...,16,48,1,3,0,2,3,879015,267,274
